In [216]:
import numpy as np
import pandas as pd

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#stats
import statsmodels.api as sm
from statsmodels.api import tsa



In [217]:
air_traffic = pd.read_csv('/content/drive/MyDrive/projects/airline_TSA/air_traffic.csv')
air_traffic.head()

,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12
3,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07
4,2003,6,"52,209,516","5,411,504","57,621,020","798,351","58,225","856,576","44,492,972","13,918,185","58,411,157","56,555,517","17,191,579","73,747,096",78.67,80.96,79.20


In [218]:
#lets combine year and month and add a day as 1st of each month to have a full date column
air_traffic['Date']= pd.to_datetime(air_traffic.assign(Day=1).loc[:, ['Year', 'Month', 'Day']])
air_traffic.head()


,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF,Date
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20,2003-01-01
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31,2003-02-01
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12,2003-03-01
3,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07,2003-05-01
4,2003,6,"52,209,516","5,411,504","57,621,020","798,351","58,225","856,576","44,492,972","13,918,185","58,411,157","56,555,517","17,191,579","73,747,096",78.67,80.96,79.20,2003-06-01


In [219]:
#since we are only focusing overall flight stats, we will remove all domestic and international individual columns
air_traffic_removed= air_traffic.drop(columns=['Dom_Pax','Int_Pax','Dom_Flt','Int_Flt','Dom_RPM','Int_RPM','Dom_ASM','Int_ASM','Dom_ASM','Year','Month','Dom_LF','Int_LF','LF','ASM'])

In [220]:
#let's rename the columns for better understanding
air_traffic_renamed= air_traffic_removed.rename(columns={'Pax':'Passengers','Flt':'Flights','RPM':'Revenue Per Mile'})
air_traffic_renamed.head()

,Passengers,Flights,Revenue Per Mile,Date
0,"47,938,280","842,827","49,097,402",2003-01-01
1,"45,412,146","741,610","44,863,907",2003-02-01
2,"55,001,313","856,120","54,341,633",2003-03-01
3,"53,763,186","844,662","52,576,960",2003-05-01
4,"57,621,020","856,576","58,411,157",2003-06-01


In [221]:
# EDA
air_traffic_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Passengers        233 non-null    object        
 1   Flights           233 non-null    object        
 2   Revenue Per Mile  233 non-null    object        
 3   Date              243 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 7.7+ KB


In [222]:
# let's update the index of dataframe to date,
# To perform any TSA, keeping date to index enables a lot of flexibility
air_traffic_renamed= air_traffic_renamed.set_index('Date')
air_traffic_renamed.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 243 entries, 2003-01-01 to 2023-09-01
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Passengers        233 non-null    object
 1   Flights           233 non-null    object
 2   Revenue Per Mile  233 non-null    object
dtypes: object(3)
memory usage: 7.6+ KB


In [223]:
# let's convert the values to appropriate datatype
air_traffic_renamed['Passengers'] = air_traffic_renamed['Passengers'].str.replace(',', '', regex=False).astype('Int64')
air_traffic_renamed['Flights'] = air_traffic_renamed['Flights'].str.replace(',', '', regex=False).astype('Int64')
air_traffic_renamed['Revenue Per Mile'] = air_traffic_renamed['Revenue Per Mile'].str.replace(',', '', regex=False).astype('Int64')




In [224]:
air_traffic_renamed.head()

,Passengers,Flights,Revenue Per Mile
Date,,,
2003-01-01,47938280,842827,49097402
2003-02-01,45412146,741610,44863907
2003-03-01,55001313,856120,54341633
2003-05-01,53763186,844662,52576960
2003-06-01,57621020,856576,58411157


In [225]:
# we can see that there are null rows in our data
# let's visualize how it looks at the begining
fig = px.line(
    air_traffic_renamed,
    x=air_traffic_renamed.index,
    y=air_traffic_renamed.columns,
    labels={'x': 'Date', 'value': 'Air Traffic Data'},
    title='Traffic Data from 2003 to 2023'
)


fig.update_layout(
    yaxis_title=' air traffic data',
    title='Traffic Data from 2003 to 2023'

)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [226]:
# Before doing any update to the nulls let's examine if there are any missing dates
# let's verify the total number of months between 2003-01 and 2023-09
# how we will do it is see the total number of months between those two range and compare with our data
first_day = air_traffic_renamed.index.min()
last_day= air_traffic_renamed.index.max()
total_months = (last_day.year - first_day.year)*12 + (last_day.month-first_day.month) + 1

air_traffic_renamed.shape[0]== total_months

False

In [238]:
# since the dates are not complete, we will first fill in the date date indexes
# let's get the date range from first day to last day
full_dates_range= pd.date_range(start=first_day,end=last_day,freq='MS')

In [239]:
# let's compare the difference with our dataset index dates
full_dates_range.difference(air_traffic_no_april.index)

DatetimeIndex(['2003-04-01', '2004-04-01', '2005-04-01', '2006-04-01',
               '2007-04-01', '2008-04-01', '2009-04-01', '2010-04-01',
               '2011-04-01', '2012-04-01', '2013-04-01', '2014-04-01',
               '2015-04-01', '2016-04-01', '2017-04-01', '2018-04-01',
               '2019-04-01', '2020-04-01', '2021-04-01', '2022-04-01',
               '2023-04-01'],
              dtype='datetime64[ns]', freq=None)

In [240]:
# we are missing the above dates in our dataset
# Now let's add these dates to our dateindex dataset by reindexing

air_traffic_dates_updated= air_traffic_no_april.reindex(full_dates_range)

air_traffic_dates_updated.shape

(249, 3)

In [241]:
# dates have been added , let's check the data
air_traffic_dates_updated.isna().sum()

,0
Passengers,21
Flights,21
Revenue Per Mile,21


In [242]:
# To impute the missing values, we will use one of the following imputation technique
# ffill - fill the values based on the last available data
# bfill - fill the values based on the next available data
# interpolation - fill the values based on average of first two available data forward and backward

fig = px.line(
    air_traffic_dates_updated,
    x=air_traffic_dates_updated.index,
    y=air_traffic_dates_updated['Revenue Per Mile'],
    labels={'x': 'Date', 'value': 'Air Traffic Data'},
    title='Revenue per mile from 2003 to 2023'
)


fig.update_layout(
    yaxis_title=' air traffic data',

)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [243]:
# in those missing values, we will try to see how the graph looks with imputation using ffill and bfill
# we can choose the best fill after seeing the graph which looks more consistent
air_traffic_impute_test = air_traffic_dates_updated.copy()
air_traffic_impute_test['bfill']= air_traffic_impute_test['Revenue Per Mile'].bfill()
air_traffic_impute_test['ffill']= air_traffic_impute_test['Revenue Per Mile'].ffill()


In [244]:
# let's compare both fills in graph
fig = px.line(
    air_traffic_impute_test,
    x=air_traffic_impute_test.index,
    y=['Revenue Per Mile', 'bfill', 'ffill'],
    labels={'x': 'Date', 'value': 'Air Traffic Data'},
    title='Revenue per mile from 2003 to 2023'
)

fig.update_layout(
    yaxis_title='Air Traffic Data',
)

fig.update_xaxes(rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [245]:
# Looking at the graph the better would be ffill , seeing the consistency
air_traffic_final = air_traffic_dates_updated.fillna(method='ffill')
air_traffic_final.isna().sum()

<ipython-input-245-8d673472d6c1>:2: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,0
Passengers,0
Flights,0
Revenue Per Mile,0


In [246]:
# comparing revenue of each month to the overall average revenue

# monthly mean of individual months over the whole time period
monthly_mean = air_traffic_final.groupby(air_traffic_final.index.month_name())['Revenue Per Mile'].mean()

# relative deviation from whole mean
monthly_mean_diff = (monthly_mean - monthly_mean.mean())/monthly_mean

# monthly names in order
monthly_names = pd.date_range(start='2000-01',freq='MS', periods=12).month_name()

monthly_mean_diff = monthly_mean_diff.loc[monthly_names,]

monthly_mean_diff.to_frame().T

,January,February,March,April,May,June,July,August,September,October,November,December
Revenue Per Mile,-0.115451,-0.202047,0.016032,0.016032,0.016547,0.086834,0.141037,0.107511,-0.054735,-0.014675,-0.081117,-0.016449


In [248]:
# To actually visualize the rate of changes, we can plot a graph
fig = px.bar(monthly_mean_diff)

fig.update_layout(
    title ='Monthly deviation from mean revenue'
)
fig.show()

In [ ]:
# From the data and the graph, it is clearly visible that June,July,August are the most busy months
# while january and febraury have the lowest air travel